In [1]:

import pandas as pd
import polars as pl
from utils import add_districts

main_folder = 'snapshots'
# Stations master file
#URL: https://opendata-ajuntament.barcelona.cat/data/es/dataset/informacio-estacions-bicing
stations_master_file = main_folder+ '/2024_05_Maig_BicingNou_INFORMACIO.csv'

# Polars
#stations_master = pl.read_csv(stations_master_file, ignore_errors = True)

# Pandas
stations_master = pd.read_csv(stations_master_file, dtype={'short_name': str}).sort_values(by='station_id')
stations_master['last_updated'] = pd.to_datetime(stations_master['last_updated'])
stations_master = stations_master.sort_values('last_updated').groupby('station_id').last()

/var/folders/md/jgs9sl817gxfmk03qhzt6bk80000gq/T/ipykernel_58835/2348295205.py:14: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  stations_master = pd.read_csv(stations_master_file, dtype={'short_name': str}).sort_values(by='station_id')


In [2]:
import matplotlib.pyplot as plt
import plotly.express as px

capacity_counts = stations_master.groupby('capacity').size().reset_index(name='number_of_stations')
capacity_counts.sort_values('capacity', inplace=True)
fig = px.bar(
    capacity_counts,
    x='capacity',
    y='number_of_stations',
    title='Number of Stations by Capacity',
    labels={'capacity': 'Capacity', 'number_of_stations': 'Number of Stations'},
    color='number_of_stations',  # Colorear las barras basadas en el número de estaciones
    template='plotly_white'  # Estilo del gráfico
)

fig.update_layout(xaxis_title='Capacity', yaxis_title='Number of Stations')
fig.show()

In [3]:
# Agrupar por 'postcode' y sumar las capacidades para cada código postal
postcode_capacity = stations_master.groupby('post_code')['capacity'].sum().reset_index()
postcode_capacity.sort_values('capacity', inplace=True)

fig = px.bar(
    postcode_capacity,
    x='post_code',
    y='capacity',
    title='Total Capacity by Postcode',
    labels={'postcode': 'Postcode', 'capacity': 'Total Capacity'},
    color='capacity',  # Colorear las barras basadas en la capacidad total
    template='plotly_white'  # Estilo del gráfico
)

fig.update_layout(xaxis_title='Postcode', yaxis_title='Total Capacity')
fig.show()


In [4]:
# Agrupar por 'district' y sumar las capacidades para cada distrito
district_capacity = add_districts(stations_master)
district_capacity = district_capacity.groupby('district')['capacity'].sum().reset_index()
district_capacity.sort_values('capacity', inplace=True)

fig = px.bar(
    district_capacity,
    x='district',
    y='capacity',
    title='Total Capacity by District',
    labels={'district': 'District', 'capacity': 'Total Capacity'},
    color='capacity',  # Colorear las barras basadas en la capacidad total
    template='plotly_white'  # Estilo del gráfico
)

fig.update_layout(xaxis_title='Postcode', yaxis_title='Total Capacity')
fig.show()
